# A Feature Selection Approach using QCi's Dirac-1


## Introduction
In machine learning problems, we often have to start with a large number of features. We need a feature selection technique that can discover a relatively small subset of the most relevant features. In what follows we present a tutorial on using QCi's technology to select a set of features by minimzing their inter-correlation. This approach can be used with any unsupervised machine learning approach such as anomaly detection and clustering algorithms. 

## Methodology

Let us have a dataset with $N$ samples and $d$ features, represented by a $N \times d$ matrix $F$. Moreover, let us each column of $F$ by $\bf{f}_{j}$ where $j \in {1,...,d}$, representing all samples for feature $j$. We can choose a subset features of size $d^\prime$ ($d^\prime < d$) such that the inter-correlation of the subset is minimal. We have

$$
\begin{equation}
  \label{eqn:corr-method-min-1}
  \min_{\bf{x}} \sum_{i=1}^{d} \sum_{j=1}^{d} C_{ij} x_{i} x_{j}
\end{equation}
$$

where

$$
\begin{equation}
  \label{eqn:corr-matrix}
  C_{ij} = |corr(\bf{f_{i}}, \bf{f_{j}})|
\end{equation}
$$

where $corr$ denotes a correlation function such as the Pearson correlation, and $x_{i} \in \{0, 1\}$ is a binary variable indicating inclusion or exclustion of feature $i$. Obviously, $C_{ii} = 1$. The above minimization problem is subject to a constraint,

$$
\begin{equation}
  \label{eqn:corr-method-constriant}
  \sum_{i=1}^{d} x_{i} = d^\prime
\end{equation}
$$

We can exclude the diagonal elements of $C$ as they always add up to
$d^\prime$. In the matrix form we have,

$$
\begin{equation}
  \label{eqn:corr-method-min-2}
  \min_{\bf{x}} \bf{{x}^{T}} (C - I) \bf{{x}}
\end{equation}
$$

subject to the above constraint. Note that $I$ is a $d \times d$ identity matrix. Note too that we have assumed that the reduced dimension $d^\prime$ is assumed to be given in the above approach. 

## Medicare Prescriotion Data

We impelemented this approach using a publically available set of data on prescription of opioids in the United States. 

We start by imprting the libraries, setting some parameters

https://www.cms.gov/data-research/statistics-trends-and-reports/medicare-provider-utilization-payment-data/part-d-prescriber

## Clean data

In [30]:
import pandas as pd

# Input                                                                                       
INP_FILE = "Medicare_Provider_Utilization_and_Payment_Data__Part_D_Prescriber_Summary_Table_CY2014__50001-NNN__ANON.csv"
OUT_FILE  = "cleaned_medicare_data.csv"

CON_VARS = [
    "total_claim_count",
    "total_30_day_fill_count",
    "total_drug_cost",
    "total_day_supply",
    "bene_count",
    "total_claim_count_ge65",
    "total_30_day_fill_count_ge65",
    "total_drug_cost_ge65",
    "total_day_supply_ge65",
    "bene_count_ge65",
    "brand_claim_count",
    "brand_drug_cost",
    "generic_claim_count",
    "generic_drug_cost",
    "other_claim_count",
    "other_drug_cost",
    "mapd_claim_count",
    "mapd_drug_cost",
    "pdp_claim_count",
    "pdp_drug_cost",
    "lis_claim_count",
    "lis_drug_cost",
    "nonlis_claim_count",
    "nonlis_drug_cost",
    "opioid_claim_count",
    "opioid_drug_cost",
    "opioid_day_supply",
    "opioid_bene_count",
    "opioid_prescriber_rate",
    "antibiotic_claim_count",
    "antibiotic_drug_cost",
    "antibiotic_bene_count",
    "hrm_claim_count_ge65",
    "hrm_drug_cost_ge65",
    "hrm_bene_count_ge65",
    "antipsych_claim_count_ge65",
    "antipsych_drug_cost_ge65",
    "antipsych_bene_count_ge65",
    "average_age_of_beneficiaries",
    "beneficiary_age_less_65_count",
    "beneficiary_age_65_74_count",
    "beneficiary_age_75_84_count",
    "beneficiary_age_greater_84_count",
    "beneficiary_female_count",
    "beneficiary_male_count",
    "beneficiary_race_white_count",
    "beneficiary_race_black_count",
    "beneficiary_race_asian_pi_count",
    "beneficiary_race_hispanic_count",
    "beneficiary_race_nat_ind_count",
    "beneficiary_race_other_count",
    "beneficiary_nondual_count",
    "beneficiary_dual_count",
    "beneficiary_average_risk_score",
]

VALID_PROVIDER_MI = [
    "A",
    "M",
    "J",
    "L",
    "R",
    "S",
    "E",
    "D",
    "C",
    "B",
    "K",
    "P",
    "W",
    "H",
    "T",
    "G",
    "F",
    "N",
    "V",
    "I",
    "O",
    "Y",
    "Z",
    "U", 
    "Q",
    "X",
]

VALID_GEN = ["F", "M", "Other", "Unknown"]

VALID_ENTITIES = ["I", "O"]

VALID_DESC_FLAGS = ["S", "T"]

VALID_ENROLLS = ["E", "N", "O"]

# Some utilities                                                                              
def convert_to_float(x):
    try:
        return float(x)
    except:
        return None
    
def convert_to_int(x):
    try:
        return int(float(x))
    except:
        return None

# Read data                                                                                   
df = pd.read_csv(INP_FILE, on_bad_lines = "skip", low_memory=False)
    
# Clean categorical variables                                                                 
df["nppes_provider_mi"] = df["nppes_provider_mi"].fillna("Unknown")
df["nppes_provider_mi"] = df["nppes_provider_mi"].apply(
    lambda x: x if x in VALID_PROVIDER_MI else "Unknown"
)

df["nppes_credentials"] = df["nppes_credentials"].fillna("Unknown")
df["nppes_credentials"] = df["nppes_credentials"].apply(
    lambda x: str(x).replace(".", "")
)

cred_hash = {
    "MEDICAL DOCTOR": "MD",
    "NURSE PRACTITIONER": "NP",
}
df["nppes_credentials"] = df["nppes_credentials"].apply(
    lambda x: cred_hash[x] if x in cred_hash else x,
)

df["nppes_provider_gender"] = df["nppes_provider_gender"].fillna("Unknown")
df["nppes_provider_gender"] = df["nppes_provider_gender"].apply(
    lambda x: x if x in VALID_GEN else "Other",
)

df["nppes_entity_code"] = df["nppes_entity_code"].apply(
    lambda x: x if x in VALID_ENTITIES else "Unknown",
)

df["nppes_provider_zip5"] = df["nppes_provider_zip5"].fillna("Unknown")

df["nppes_provider_country"] = df["nppes_provider_country"].apply(
    lambda x: "US" if x == "US" else "Other",
)

df["description_flag"] = df["description_flag"].apply(
    lambda x: x if x in VALID_DESC_FLAGS else "Unknown",
)

df["medicare_prvdr_enroll_status"] = df["medicare_prvdr_enroll_status"].apply(
    lambda x: x if x in VALID_ENROLLS else "Unknown",
)


# Treat missing beneficiary count as it cannot be zero                                        
df["bene_count"] = df["bene_count"].apply(
    convert_to_int
).fillna(-1)

tmp_df = df.groupby(
    "specialty_description", as_index=False,
)["bene_count"].mean()

bene_count_hash = dict(
    zip(
        tmp_df["specialty_description"],
        tmp_df["bene_count"],
    )
)
df["bene_count"] = df.apply(
    lambda x: x["bene_count"] if x[
        "bene_count"
    ] > 0 else bene_count_hash[
        x["specialty_description"]
    ],
    axis=1,
)

# Treat continuous variables                                                                  
for item in CON_VARS:
    df[item] = df[item].apply(
        convert_to_float
    ).fillna(0.0)

# Filter out invalid states                                                                   
df  = df[
    ~df["nppes_provider_state"].isin(
        ["XX", "E", "N", "S"]
    )
]

# Output                                                                                      
df.to_csv(OUT_FILE, index=False)


## Generate features

In [31]:
import pandas as pd

# Input                                                                                       
INP_FILE = "cleaned_medicare_data.csv"
OUT_FILE  = "medicare_features.csv"

# Set some parameters                                                                         
CAT_VARS = [
    "nppes_provider_mi",
    #"nppes_credentials", # This is rather messy, so ignoring it.                             
    "nppes_provider_gender",
    "nppes_entity_code",
    "nppes_provider_city",
    "nppes_provider_zip5",
    #"nppes_provider_country", # Almost all cases are US                                      
    "specialty_description",
    "medicare_prvdr_enroll_status",
    "nppes_provider_state",
]

CON_VARS = [
    "total_claim_count",
    "total_30_day_fill_count",
    "total_drug_cost",
    "total_day_supply",
    "bene_count",
    "total_claim_count_ge65",
    "total_30_day_fill_count_ge65",
    "total_drug_cost_ge65",
    "total_day_supply_ge65",
    "bene_count_ge65",
    "brand_claim_count",
    "brand_drug_cost",
    "generic_claim_count",
    "generic_drug_cost",
    "other_claim_count",
    "other_drug_cost",
    "mapd_claim_count",
    "mapd_drug_cost",
    "pdp_claim_count",
    "pdp_drug_cost",
    "lis_claim_count",
    "lis_drug_cost",
    "nonlis_claim_count",
    "nonlis_drug_cost",
    "opioid_claim_count",
    "opioid_drug_cost",
    "opioid_day_supply",
    "opioid_bene_count",
    "antibiotic_claim_count",
    "antibiotic_drug_cost",
    "antibiotic_bene_count",
    "hrm_claim_count_ge65",
    "hrm_drug_cost_ge65",
    "hrm_bene_count_ge65",
    "antipsych_claim_count_ge65",
    "antipsych_drug_cost_ge65",
    "antipsych_bene_count_ge65",
    "average_age_of_beneficiaries",
    "beneficiary_age_less_65_count",
    "beneficiary_age_65_74_count",
    "beneficiary_age_75_84_count",
    "beneficiary_age_greater_84_count",
    "beneficiary_female_count",
    "beneficiary_male_count",
    "beneficiary_race_white_count",
    "beneficiary_race_black_count",
    "beneficiary_race_asian_pi_count",
    "beneficiary_race_hispanic_count",
    "beneficiary_race_nat_ind_count",
    "beneficiary_race_other_count",
    "beneficiary_nondual_count",
    "beneficiary_dual_count",
    "beneficiary_average_risk_score",
]

# Read and clean data                                                                         
df = pd.read_csv(INP_FILE, low_memory=False)

# Embed categorical features                                                                  
embedded_cat_features = []
for item in CAT_VARS:
    tmp_df = df.groupby(item, as_index=False).agg(
        {
            "opioid_claim_count": "mean",
            "opioid_drug_cost": "mean",
            "opioid_day_supply": "mean",
            "opioid_bene_count": "mean",
            "opioid_prescriber_rate": "mean",
        }
    ).rename(
        columns={
            "opioid_claim_count": "%s_opioid_claim_count" % item,
            "opioid_drug_cost": "%s_opioid_drug_cost" % item,
            "opioid_day_supply": "%s_opioid_day_supply" % item,
            "opioid_bene_count": "%s_opioid_bene_count" % item,
            "opioid_prescriber_rate": "%s_opioid_prescriber_rate" % item,
        }
    )

    df = df.merge(tmp_df, how="left", on=item)

    embedded_cat_features += [
        "%s_opioid_claim_count" % item,
        "%s_opioid_drug_cost" % item,
        "%s_opioid_day_supply" % item,
        "%s_opioid_bene_count" % item,
        "%s_opioid_prescriber_rate" % item,
    ]

# Drop unembedded categorical variables and some others                                       
df = df[["npi"] + CON_VARS + embedded_cat_features]

# Write features file                                                                         
df.to_csv(OUT_FILE, index=False)

## Feature Selection

In [32]:
# Import libs
import sys
import os
import time
import numpy as np
import pandas as pd

from qci_client import QciClient
from qci_client import load_json_file


# Define some parameters
FEATURES_FILE = "medicare_features.csv"
REDUCED_DIM = 10

# Read features
df = pd.read_csv(FEATURES_FILE, low_memory=False)

We now print the feature names and get the total count of features in the dataset,

In [33]:
feature_names = list(set(df.columns) - {"npi"})

orig_dim = len(feature_names)

print(
    "Original dimension is %d; reduced dimension will be %d" % (
        orig_dim,
        REDUCED_DIM,
    )
)


Original dimension is 93; reduced dimension will be 10


We should now create the objective matrix $C$,

In [34]:
# Generate the objective matrix
X = np.array(df[feature_names])

C = abs(np.corrcoef(X, rowvar=False))

# Make correlation symmetric to machine precision
C = 0.5 * (C + C.transpose())

objective = C - np.eye(orig_dim)

And create the constraint matrix,

In [41]:
# Generate the constraint
cons_lhs = np.zeros(shape=(orig_dim), dtype=np.float64)
for i in range(orig_dim):
    cons_lhs[i] = 1.0

cons_lhs = np.array([cons_lhs])
cons_rhs = [REDUCED_DIM]

We now solve the above quadratic binary problem using QCi's Dirac-1,

In [46]:
print(cons_lhs)
# Create json objects                                                                     
objective_json = {}
objective_json["data"] = []
for i in range(orig_dim):
    for j in range(orig_dim):
        if objective[i][j] == 0:
            continue
        objective_json["data"].append(
            {"i": i, "j": j, "val": objective[i][j]}
        )
        
objective_json["file_name"] = "objective_tutorial.json"
objective_json["num_variables"] = orig_dim
objective_json["file_type"] = "objective"

# Create constraint jasons
constraint_json = {}
constraint_json["data"] = []
for i in range(cons_lhs.shape[0]):
    for j in range(cons_lhs.shape[1]):
        if cons_lhs[i][j] == 0:
            continue
        constraint_json["data"].append(
            {"i": i, "j": j, "val": cons_lhs[i][j]}
        )
        
constraint_json["file_name"] = "constraints_tutorial.json"
constraint_json["num_constraints"] = len(cons_rhs)
constraint_json["num_variables"] = orig_dim
constraint_json["file_type"] = "constraints"

rhs_json = {}
rhs_json["data"] = cons_rhs
rhs_json["file_name"] = "rhs_tutorial.json"
rhs_json["num_constraints"] = len(cons_rhs)
rhs_json["file_type"] = "rhs"
    
# Solve the optimizzation problem
qci = QciClient()

#response_json = qci.upload_file(objective_json)
#objective_file_id = response_json["file_id"]

response_json = qci.upload_file(constraint_json)
constraint_file_id = response_json["file_id"]

response_json = qci.upload_file(rhs_json)
rhs_file_id = response_json["file_id"]

# Setup job json
job_json = {
    "job_name": "tutorial_eqc1",
    "job_tags": ["tutorial_eqc1"],
    "params": {
        "sampler_type": "csample", #"eqc1",                                             
        "n_samples": 20,
        "alpha": 1.0,
    },
}
job_json["constraints_file_id"] = constraint_file_id
job_json["objective_file_id"] = objective_file_id
job_json["rhs_file_id"] = rhs_file_id

# Run the job
job_response_json = qci.process_job(
    job_body=job_json, job_type="sample-constraint",
)

print(job_response_json)

if job_response_json["job_info"]["details"]["status"] == "COMPLETED":
    results = job_response_json["results"]
    energies = results["energies"]
    samples = results["samples"]
    is_feasibles = results["is_feasible"]
else:
    assert False, job_response_json["job_info"]["results"]["error"]
        
if True:
    print("Energies:", energies)        

# Pick a feasible solution with lowest energy                                             
# The sample solutions are sorted by energy                                               
sol = None
for i, item in enumerate(samples):
    sol = item
    is_feasible = is_feasibles[i]

    if is_feasible:
        break

if not is_feasible:
    print("Solution is not feasible!")

assert sol is not None, "No feasible solution found!"

assert len(sol) == orig_dim, "Inconsistent solution size!"

assert sum(sol) == REDUCED_DIM, "Solution is not feasible!"

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


TypeError: Subscripted generics cannot be used with class and instance checks

Finally, we can print the list of selected variables,

In [ ]:
selected_vars = []
for i in range(orig_dim):
    if sol[i] > 0:
        selected_vars.append(feature_names[i])

print(selected_vars)      